In [1]:
#prodigal -i Zymo-GridION-EVEN-3Peaks-R103-merged.fasta -a R103_protein -d R103_DNA  
#diamond blastp -d /big7_disk/guocheng107/Zymo/card_db.dmnd -q R103_protein -k 1 --more-sensitive -f 6 qseqid salltitles pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen qseq sseq -o R103_diamond.m8


# awk '{print $2}' R103_diamond.m8 | cut -d '|' -f 3 > ARO   *切出ARO
# vim ARO
# :1,$s/ARO://g
# paste ARO R103_diamond.m8 > R103_diamond
# awk 'FNR==NR{a[$1]=$2; next}{ print $0, a[$1]}' aro_index R103_diamond >R103_diamond_ARG    *加入ARG


# awk ' { tot[$1]++ } END { for (i in tot) print tot[i],i } ' R103_diamond_ARG | awk '{print $2 "\t"$1}' > test  *算覆蓋次數
# awk 'NR==FNR{a[$1]=$2;next}$1 in a{print $1,a[$1]}' test R103_diamond_ARG >test2 
# paste test2 R103_diamond_ARG > all                                               


# awk '{print $1 "\t" $20 "\t" $2 "\t" $6 "\t" $14 "\t" $15 "\t" ($11-$10+1)/$16"\t" ($13-$12+1)/$17}' all > all_feature  *選出需要的feature
# #(ARO ARG ARG_depth ident evalue bitscore query_width ARG_width) *selection name


# cat all_feature | tr -d '\r'  | sort -k1n > all_feature2  *消除M^(換行)，依ARO排序


# sort -u -d -k1n,1 -k4n,4r all_feature2 |sort -u -d -k1n,1> uniq_ident   *選出feature max value
# awk '{$5=sprintf("%.32f",$5)}7' all_feature2 | sort -u -d -k1n,1 -k5n,5r |sort -u -d -k1n,1> uniq_evalue
# sort -u -d -k1n,1 -k6n,6r all_feature2 |sort -u -d -k1n,1> uniq_bitscore
# sort -u -d -k1n,1 -k7n,7r all_feature2 |sort -u -d -k1n,1> uniq_querywidth
# sort -u -d -k1n,1 -k8n,8r all_feature2 |sort -u -d -k1n,1> uniq_ARGwidth
# paste <(awk '{print $1 "\t" $2 "\t" $3 "\t" $4}' uniq_ident ) <(awk '{print $5}' uniq_evalue ) <(awk '{print $6}' uniq_bitscore) <(awk '{print $7}' uniq_querywidth) <(awk '{print $8}' uniq_ARGwidth) >all_max_feature


# awk 'FNR==NR{a[$1]=$2; next}{ print $0, a[$1]}' groundtruth all_max_feature >label_feature   *加入label
# awk '!$9{$9=0}1' label_feature > R103_feature


# ( echo -e "ARO ARG ARG_depth ident evalue bitscore query_width ARG_width label"; cat R103_feature )> R103_final_feature     *加入header
# awk '{print $1 "\t" $2 "\t" $3 "\t" $4 "\t" $5 "\t" $6 "\t" $7 "\t" $8"\t" $9}' R103_final_feature > R103_feature_final

In [2]:
import pandas as pd
df = pd.read_csv(r'R103_feature_final',sep='\t')

df['query_width'] = df['query_width']*100
df['ARG_width'] = df['ARG_width']*100
#df

X = df[['ARG_depth','ident','evalue','bitscore','query_width','ARG_width']]
y = df['label']
Z = df['ARO']
print(X.head())
print(y.head())

   ARG_depth  ident   evalue  bitscore  query_width  ARG_width
0         35   54.3  0.00064      89.0      97.0000    64.1399
1         15   53.2  0.00053     125.6      98.1982    63.5569
2         47   58.3  0.00076      92.0      97.9381    69.0058
3       4239  100.0  0.00099     522.7     100.0000   100.0000
4       1911   84.2  0.00099     401.0     100.0000    98.6395
0    0
1    0
2    1
3    1
4    1
Name: label, dtype: int64


In [3]:
import pandas as pd
import numpy as np

#Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3 ,random_state=99)

# train set label proportion
print(y_train.value_counts(normalize=True))

0    0.638336
1    0.361664
Name: label, dtype: float64


In [4]:
import warnings 
warnings.filterwarnings('ignore')

#import Scaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = RobustScaler()
scaler2 = StandardScaler()



X_train['ARG_depth'] = scaler.fit_transform(X_train[['ARG_depth']])
X_train['evalue'] = scaler2.fit_transform(X_train[['evalue']])
X_train['bitscore'] = scaler.fit_transform(X_train[['bitscore']])
X_train['ident'] = scaler2.fit_transform(X_train[['ident']])
X_train['query_width'] = scaler.fit_transform(X_train[['query_width']])
X_train['ARG_width'] = scaler2.fit_transform(X_train[['ARG_width']])
#X_train['ARGdepth/truthcover'] = scaler3.fit_transform(X_train[['ARGdepth/truthcover']])
#X_train['pass_bitscore'] = scaler4.fit_transform(X_train[['pass_bitscore']])
#X_train['mismatch_Min'] = scaler3.fit_transform(X_train[['mismatch_Min']])


X_test['ARG_depth'] = scaler.fit_transform(X_test[['ARG_depth']])
X_test['evalue'] = scaler2.fit_transform(X_test[['evalue']])
X_test['bitscore'] = scaler.fit_transform(X_test[['bitscore']])
X_test['ident'] = scaler2.fit_transform(X_test[['ident']])
X_test['query_width'] = scaler.fit_transform(X_test[['query_width']])
X_test['ARG_width'] = scaler2.fit_transform(X_test[['ARG_width']])
#X_test['ARGdepth/truthcover'] = scaler3.fit_transform(X_test[['ARGdepth/truthcover']])
#X_test['pass_bitscore'] = scaler4.fit_transform(X_test[['pass_bitscore']])
#X_test['mismatch_Min'] = scaler3.fit_transform(X_test[['mismatch_Min']])


print(X_train)

     ARG_depth     ident    evalue  bitscore  query_width  ARG_width
453   4.140625  1.257881  0.881126  1.884044     0.210588   0.059144
500   1.381250  1.257881  0.358202  1.588206     0.139734  -1.203641
565  -0.181250 -1.352766 -1.816058 -0.080278    -0.224045  -0.217786
166  -0.165625 -1.475716 -1.727987 -0.228940    -0.357186  -1.310371
46    0.478125  1.257881  0.550858  0.742319     0.116560   1.259164
..         ...       ...       ...       ...          ...        ...
713  -0.178125 -1.524896 -0.660122 -0.345391    -8.417409  -1.340173
168   0.225000  1.257881 -1.265612  0.259663     0.042596  -0.514567
185   2.440625  1.257881  0.715992  1.244301     0.210588   1.259164
35   -0.128125 -0.356867  0.881126 -0.341923    -2.613540  -1.328373
641  25.615625  1.257881  0.908648  1.590188     0.210588   1.259164

[553 rows x 6 columns]


In [5]:
import warnings 
warnings.filterwarnings('ignore')
# Feature selection and Model selection.
# Imports for pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression

# Create pipeline
pipeline = make_pipeline(\

                         SelectKBest(mutual_info_classif),
                         LogisticRegression())


# Model validation.
from sklearn.model_selection import GridSearchCV


param_grid = {
    'selectkbest__k': [1,2,3,4,5,6],
    'logisticregression__solver': [ 'newton-cg','lbfgs','liblinear', 'sag', 'saga'],
    'logisticregression__class_weight': [None,'balanced'],
    'logisticregression__C': [0.0001,0.001,0.01,0.1,1,100,500,1000]
    
    
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring='accuracy', verbose=99)

gridsearch.fit(X_train, y_train)

Fitting 10 folds for each of 480 candidates, totalling 4800 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=1, score=0.643, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=1 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=1, score=0.643, total=   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, select

[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=3, score=0.643, total=   0.1s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    1.4s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=3 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=3, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    1.5s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=3 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=3, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    1.6s remaining:    0.0s
[CV] logisticregression__C

[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=5, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  47 out of  47 | elapsed:    2.9s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=5 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=5, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    3.0s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=5 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=newton-cg, selectkbest__k=5, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:    3.0s remaining:    0.0s
[CV] logisticregression__C

[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=2, score=0.643, total=   0.1s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:    4.3s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=2 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=2, score=0.643, total=   0.1s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    4.4s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=2 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=2, score=0.643, total=   0.1s
[Parallel(n_jobs=1)]: Done  73 out of  73 | elapsed:    4.5s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticreg

[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=4, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  95 out of  95 | elapsed:    5.7s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=4 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=4, score=0.655, total=   0.1s
[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:    5.8s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=4 
[CV]  logisticregression__C=0.0001, logisticregression__class_weight=None, logisticregression__solver=lbfgs, selectkbest__k=4, score=0.636, total=   0.1s
[Parallel(n_jobs=1)]: Done  97 out of  97 | elapsed:    5.9s remaining:    0.0s
[CV] logisticregression__C=0.0001, logisticreg

...

[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3, score=0.800, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3, score=0.909, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3, score=0.927, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=3, score=0.909, total=   0.1s
[CV] l

[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6, score=0.909, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6, score=0.891, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6, score=0.909, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=sag, selectkbest__k=6, score=0.927, total=   0.1s
[CV] l

[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3, score=0.909, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3, score=0.873, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3, score=0.945, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=3, score=0.891, total=   0.1s

[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=6, score=0.927, total=   0.1s
[CV] logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=6 
[CV]  logisticregression__C=1000, logisticregression__class_weight=balanced, logisticregression__solver=saga, selectkbest__k=6, score=0.945, total=   0.1s
[Parallel(n_jobs=1)]: Done 4800 out of 4800 | elapsed:  4.8min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('selectkbest',
                                        SelectKBest(k=10,
                                                    score_func=<function mutual_info_classif at 0x7fe3039832f0>)),
                                       ('logisticregression',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=100,
                                                           multi_class='warn',
            

In [6]:
# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

# Which features were selected
selector = gridsearch.best_estimator_.named_steps['selectkbest']
#all_names = X_train.columns
all_names = X.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Cross Validation Score: 0.9041591320072333
Best Parameters: {'logisticregression__C': 1, 'logisticregression__class_weight': 'balanced', 'logisticregression__solver': 'newton-cg', 'selectkbest__k': 6}
Features selected:
ARG_depth
ident
evalue
bitscore
query_width
ARG_width

Features not selected:


In [7]:
#Get the best model and check it against test data set.
import sklearn.metrics as metrics
# Predict with X_test features
y_pred = gridsearch.predict(X_test)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

print(classification_report(y_test,y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

from nltk import ConfusionMatrix 
print (ConfusionMatrix(list(y_test), list(y_pred)))

#import numpy as np
#test_result = pd.concat( [pd.DataFrame(np.array(X_test)), pd.DataFrame(np.array(X_test_scaled)), pd.DataFrame(np.array(y_test).T), pd.DataFrame(np.array(y_pred).T)], axis=1 )
#test_result
#test_result.to_excel('R10.3_test_result.xlsx')

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       144
           1       0.94      0.86      0.90        93

    accuracy                           0.92       237
   macro avg       0.93      0.91      0.92       237
weighted avg       0.92      0.92      0.92       237

Accuracy: 0.9240506329113924
  |   0   1 |
--+---------+
0 |<139>  5 |
1 |  13 <80>|
--+---------+
(row = reference; col = test)

